Zadanie 1.

Zaprojektuj klasę dla $kd$-drzewa.

In [40]:
import math
# przy wykonaniu tego zadania korzystam z biblioteki pandas (również, ponieważ trochę ją poznałem),
# ponieważ dostarcza pewne ułatwiające funkcjonalności, jak:
# DataFrames - bardzo wygodny pojemnik na dane, funkcja '.iloc' i '.sort_values'.
import time

import pandas as pd
from binarytree import tree, Node

class kd_Tree:

    def __init__(self, data):
        self.data = data
        self.tree = None

    def _build(self, points, depth):

        # wyliczamy wymiar k (zakładamy, że dane otrzymujemy jako dataframe) - pandas
        k = len(points.columns)

        # ustalamy wybór kolumny (ma będą wybierane po kolei z indeksami [0], [1], [0] , [1] itd.)
        _axis = depth % k

        # wybieramy kolumnę
        _column = points.columns[_axis]
        if len(points) == 0:
            return None

        # sortyjemy DataFrame rosnąco, aby obliczyć medianę
        objects_list = points.sort_values(by= [_column], ascending= True)

        # obliczamy medianę
        median_idx = int((len(objects_list)//2))

        node = Node(int(round(objects_list.iloc[median_idx][_column],3)))
        node.left = self._build(objects_list.iloc[0:median_idx], depth+1)
        node.right = self._build(objects_list.iloc[median_idx+1:], depth+1)
        return node

    def build(self):
        self.tree = self._build(self.data, depth= 0)

test_df = pd.DataFrame(data= [[2,3],[5,4],[9,6],[4,7],[8,1],[7,2],[6,1],[10,2],[11,10],[12,5]], columns= ['X', 'Y'])

kd = kd_Tree(test_df)
kd.build()
print(kd.tree)

# Wybrałem sposób tworzenia drzewa przy użyciu Node (binarytree), więc w węzłach muszą znajdywać się pojedyńcze liczby,
# Zbiór danych, jednak pozwala jednoznacznie stwierdzić jak wygląda podział, drzewo można by było równoważnie zapisać z
# tuple'ami w węzłach, lecz sposób podany przeze mnie, jest również dopuszczony. Trzeba pamiętać, że (od góry) wartości
# występują w kolejności: wartość z kolumny X, Y, X, Y ... Więc na samej górze jest [8,1] , później [2,3] i [9,6] itd.


      ____8_____
     /          \
    3__         _6
   /   \       /  \
  7     5     12   11
 /     /     /
1     7     2



Zadanie 2.

Zaimplementuj wzorzec projektowy łańcuch odpowiedzialności na przykładzie obsługi żądania _http_ (symulacja),
w którym przed faktycznym kodem obsługi błędu ma zostać sprawdzone:
- czy użytkownik może wysłać danego typu żądania
- czy żądanie nie dotyczy pliku
- czy liczba żądań na minutę nie jest przekroczona
- czy liczba żądań na minutę nie jest przekroczona dla zalogowanego użytkownika
- czy przesłany formularz nie jest próbą `sql incjection`

In [12]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Any, Optional
from datetime import datetime
import re
import time

# Tworzę zmienną globalną role, która będzie potrzebna w handlerach do podejmowania decyzji
role = None

#Klasa, której powołane obiekty będą symulować request http
class httpSimRequest:
    def __init__(self, Type: str, File: Any, Role: str, username: str, password: str):
        self.httpReq = {
            'URL': 'https://someWebsite.com',
            'HTTP_ACCEPT': '*/*',
            'Type': Type,
            'File': File,
            'HTTP_HOST': 'MyHost',
            'HTTP_CONNECTION': 'keep-alive',
            'Minute': datetime.now(),

            # Czy użytkownik jest zalogowany sprawdzam w Role -> gdy jest zalogowany to rola to LoogedUser, jest to
            # potrzebna informacja do m.in. sprawdzenia czy liczba żądań na minutę dla zalogowanego użytkownika nie jst za duża

            'Role': Role,
            'Form': {
                'Username': username,
                'Password': password
            },
            'HTTP_ACCEPT_ENCODING': ['gzip', 'deflate']
        }
        global role
        role = self.httpReq['Role']

class Handler(ABC):

    @abstractmethod
    def set_next(self, handler: Handler) -> Handler:
        pass

    @abstractmethod
    def handle(self, request) -> Optional[str]:
        pass

class AbstractHandler(Handler):

    _next_handler: Handler = None

    def set_next(self, handler: Handler) -> Handler:
        self._next_handler = handler
        return handler

    @abstractmethod
    def handle(self, request: Any) -> str:
        if self._next_handler:
            return self._next_handler.handle(request)

        return ''

class TypeHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        # Zakładamy, że użytkownik 'user' może wysłać tylko request get, loggedUser - get, head i put a admin wszystkie
        types = ['get','head','put','post','delete','patch']
        roles = ['admin','loggedUser','user']
        if request in types:
            if role == roles[0]:
                return f"- Użytkownik może wysłać żądanie typu: {request}"

            elif role == roles[1]:
                if types.index(request) < 3:
                    return f"- Użytkownik może wysłać żądanie typu: {request}"
                else:
                    raise Exception(f"Użytkownik nie może wysłać żądania typu: {request}")

            else:
                if types.index(request) == 0:
                    return f"- Użytkownik może wysłać żądanie typu: {request}"
                else:
                    raise Exception(f"Użytkownik nie może wysłać żądania typu: {request}")

        else:
            return super().handle(request)


class FileHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == 'file':
            raise Exception("Użytkownik nie może wysłać żądania! Żądanie dotyczy pliku.")
        else:
            return super().handle(request)

# Potrzebna zmienna globalna do przechowywania wartości do przyszłych requestów
RPMCounter = {'min': -1, 'numOfReq': 0}

# Dodać exceptions, gdy nie możemy wysłać żądania
class RequestsPerMinuteHandler(AbstractHandler):
    global RPMCounter
    def handle(self, request: Any) -> str:
        roles = ['admin','loggedUser','user']

        # Zakładam, że limit żądań na minutę admin-5 loggedUser-3 user-1
        # przyjmuje tak niskie liczby dla łatwiejszego sprawdzania
        if isinstance(request, datetime):
            if RPMCounter['min'] == -1:
                RPMCounter['min'] = int(request.strftime('%M'))
                RPMCounter['numOfReq'] = 1
            elif RPMCounter['min'] == int(request.strftime('%M')):
                RPMCounter['numOfReq'] += 1
            else:
                RPMCounter['min'] = int(request.strftime('%M'))
                RPMCounter['numOfReq'] = 1
            if role == roles[0]:
                if RPMCounter['numOfReq'] < 8:
                    return f"- Użytkownik nie przekroczył jeszcze maksymalnej liczby żądań na minutę."
                else:
                    raise Exception("Użytkownik nie może wysłać żądania! Użytkownik przekroczył maksymalną liczbę żądań na minutę.")

            elif role == roles[1]:
                if RPMCounter['numOfReq'] < 6:
                    return f"- Użytkownik nie przekroczył jeszcze maksymalnej liczby żądań na minutę."
                else:
                    raise Exception("Użytkownik nie może wysłać żądania! Użytkownik przekroczył maksymalną liczbę żądań na minutę.")
            else:
                if RPMCounter['numOfReq'] < 4:
                    return f"- Użytkownik nie przekroczył jeszcze maksymalnej liczby żądań na minutę."
                else:
                    raise Exception("Użytkownik nie może wysłać żądania! Użytkownik przekroczył maksymalną liczbę żądań na minutę.")
        else:
            return super().handle(request)

# Dla ułatwienia rozwiązania zakładam, że użytkownik może podać SQL injection tylko w
# Parameters (domyślnie np. ustaleniem URL itd. zajmuje się np. strona). W tym
# rozwiązaniu w haśle nie wolno używać znaków specjalnych i spacji. Aby dokładniej wykryć SQL
# injection można posłużyć się jednym z wielu narzędzi dostępnych online jak np. py-find-injection 0.1.1

class SqlInjectionHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if type(request) == dict:
            for param in request:
                regex = re.compile("\'|\"|\(|\[|\*|=| |\\|/|;|-", re.IGNORECASE)
                if regex.search(request[param]):
                    raise Exception("Użytkownik nie może wysłać żądania! Użytkownik użył niedozwolonych znaków: możliwe sql injection.")


            return '- W parametrze nie występuje sql injection.'

        else:
            return super().handle(request)


def client_code(handler: Handler, data) -> None:

    # W przypadku TypeHandler i RequestPerMinuteHandler, upewniam się, że te klasy znają
    # wartość pola role z żądania http (będzie w nich potrzebna).
    notHandled = []
    try:
        for i in data:

            result = handler.handle(data[i])

            if result:
                print(f"{result}", end='\n\n')
            else:
                notHandled.append(i)

        print('Wysłanie żądania zakończone sukcesem!', end='\n\n')

    except Exception as e:
        print(str(e).upper())

# Aby zasymulować otwieranie pliku string (nie jest potrzebny dodatkowy plik do testu)
# Tak na prawdę, w tym przypadku nie ma to znaczenia, ponieważ musimy sprawdzić, czy operacja NIE odnosi się do pliku,
# więc będziemy tylko sprawdzać czy ma jakiekolwiek pliki w requescie (has file?) -> czy w miejscu na plik jest cokolwiek

# wywołanie:
Type = TypeHandler()
File = FileHandler()
RPM = RequestsPerMinuteHandler()
SQL = SqlInjectionHandler()

# Ustalenie kolejności wykonywania handlerów (Można wybierać tzw.
# subchain'y wpisując jako parametr w client code na 1 miejscu coś innego niż type np. File, wtedy
# subchain będzie wyglądać: File > RPM > SQL (bez type).
Type.set_next(File).set_next(RPM).set_next(SQL)

reqType = 'get'         # Typ requesta do wyboru z: 'get', 'head', 'put', 'post', 'delete', 'patch'
file = None
currRole = 'user'       # Rola jest do wyboru z: 'user', 'loggedUser', 'admin'
username = 'John123'
password = '"; select true; --"' # Próba zastosowania sql injection -> program nie może jej dopuścić
requestNumPerMinute = 1

print("Chain: Type > File > RPM > SQL\n")

print("Pierwszy test wykonania rządania, hasło jest próbą SQL-injection: '; select true; --'", end='\n\n')

for i in range(requestNumPerMinute):
    request1 = httpSimRequest(reqType, file, currRole, username, password).httpReq
    client_code(Type, request1)

time.sleep(60)

print("Wykonanie kolejnych rządań (3) po upłynięciu czasu minuty - aby liczba żądań na minutę, nie została przekroczona", end='\n\n')
password = 'password123' # haslo zmienione na poprawne, nie jest to proba sql injection
currRole = 'loggedUser' #zmieniona rola, aby uzytkownik mogl wykonac 3 rządania na minutę
requestNumPerMinute = 3

for i in range(requestNumPerMinute):
    request2 = httpSimRequest(reqType, file, currRole, username, password).httpReq
    client_code(Type, request2)

Chain: Type > File > RPM > SQL

Pierwszy test wykonania rządania, hasło jest próbą SQL-injection: '; select true; --'

- Użytkownik może wysłać żądanie typu: get

- Użytkownik nie przekroczył jeszcze maksymalnej liczby żądań na minutę.

UŻYTKOWNIK NIE MOŻE WYSŁAĆ ŻĄDANIA! UŻYTKOWNIK UŻYŁ NIEDOZWOLONYCH ZNAKÓW: MOŻLIWE SQL INJECTION.
Wykonanie kolejnych rządań (3) po upłynięciu czasu minuty - aby liczba żądań na minutę, nie została przekroczona

- Użytkownik może wysłać żądanie typu: get

- Użytkownik nie przekroczył jeszcze maksymalnej liczby żądań na minutę.

- W parametrze nie występuje sql injection.

Wysłanie żądania zakończone sukcesem!

- Użytkownik może wysłać żądanie typu: get

- Użytkownik nie przekroczył jeszcze maksymalnej liczby żądań na minutę.

- W parametrze nie występuje sql injection.

Wysłanie żądania zakończone sukcesem!

- Użytkownik może wysłać żądanie typu: get

- Użytkownik nie przekroczył jeszcze maksymalnej liczby żądań na minutę.

- W parametrze nie występuje 

Zadanie 3.

Za pomocą dowolnego wzorca projektowego zaimplementuj mechanizm sprawdzający poprawność wyrażenia postaci:
* a + b = c (poprawne),
* (a + b = c (niepoprawne),
* a + = c (niepoprawne).

In [8]:
# Używam wzorca projektowego interpreter, wzorowałem się na stronie:
# http://java.edu.pl/inne/designPatterns/15.interpreter.php, przekształciłem kod
# do języka python.

from abc import ABC, abstractmethod
import re

class Expression(ABC):
    @abstractmethod
    def interpret(self, context: str) -> bool:
        pass

class TerminalExpression(Expression):
    def __init__(self, data: str):
        self.data = data

    def interpret(self, context: str) -> bool:
        regex = re.compile(self.data)
        return regex.search(context)

class checkIfCorrect(Expression):

    def __init__(self, ex1: Expression):
        self.ex1 = ex1

    def interpret(self, context: str) -> bool:
        # Sprawdzenie czy gdy występuje znak '(' to występuje również jego zamknięcie ')'

        if not self.ex1.interpret(context):
            return False

        return True

def isCorrectExpression() -> Expression:
    regexNoBrackets = TerminalExpression("^[a-z](| )[+|-|*|/](| )[a-z](| )=(| )[a-z]$|^\([a-z]\)(| )[+|-|*|/](| )"
                                         "[a-z](| )=(| )[a-z]$|^\([a-z](| )[+|-|*|/] [a-z]\)(| )=(| )[a-z]$|^\([a-z](| )"
                                         "[+|-|*|/](| )[a-z](| )=(| )[a-z]\)$|^[a-z](| )[+|-|*|/](| )\([a-z]\)(| )=(| )[a-z]$|^[a-z]"
                                         " [+|-|*|/](| )[a-z](| )=(| )\([a-z]\)$")

    return checkIfCorrect(regexNoBrackets)

isCorrect = isCorrectExpression()

# Ulepszyłem rozwiązanie, aby uznawało wszystkie błędnie zapisane wyrażenia (jest uniwersalne, możemy sprawdzić poprawność wyrażenia
# postaci x + y = z [nawiasy mogą być wszędzie, gdzie są dozwolone), jak: a+(b=c itd. lecz poprawne jak a+(b)=c są przepuszczane,
# spacje w wyrażeniu są również opcjonalne, pokazuje kilka rozwiązań poniżej (najpierw wymagane w zadaniu).
expression1 = "a + b = c"
expression2 = "(a + b = c"
expression3 = "a + = c"
expression4 = " + b = c"
expression5 = "g +s= z"
expression6 = "a+c=d"
expression7 = "(a + b) = c"
expression8 = "(a+) = c"

print(f'Czy wyrażenie: {expression1} jest poprawne?', isCorrect.interpret(expression1))
print(f'Czy wyrażenie: {expression2} jest poprawne?', isCorrect.interpret(expression2))
print(f'Czy wyrażenie: {expression3} jest poprawne?', isCorrect.interpret(expression3))
print(f'Czy wyrażenie: {expression4} jest poprawne?', isCorrect.interpret(expression4))
print(f'Czy wyrażenie: {expression5} jest poprawne?', isCorrect.interpret(expression5))
print(f'Czy wyrażenie: {expression6} jest poprawne?', isCorrect.interpret(expression6))
print(f'Czy wyrażenie: {expression7} jest poprawne?', isCorrect.interpret(expression7))
print(f'Czy wyrażenie: {expression8} jest poprawne?', isCorrect.interpret(expression8))


Czy wyrażenie: a + b = c jest poprawne? True
Czy wyrażenie: (a + b = c jest poprawne? False
Czy wyrażenie: a + = c jest poprawne? False
Czy wyrażenie:  + b = c jest poprawne? False
Czy wyrażenie: g +s= z jest poprawne? True
Czy wyrażenie: a+c=d jest poprawne? True
Czy wyrażenie: (a + b) = c jest poprawne? True
Czy wyrażenie: (a+) = c jest poprawne? False
